In [ ]:
from build_sentence_corp import *
from prepare_input import *


In [1]:
from model import *

In [2]:
import pickle
f = open("./data/T_matrix.pickle",'rb')
_T = pickle.load(f)
f.close()

In [ ]:
with open('./data/sent_embed.dat', 'wb'):
    pickle.dump()

In [76]:
with open('./data/sample_sent_embed', 'rb') as f:
    sent_embedding_sample = pickle.load(f)

In [77]:
import numpy as np
sent_embedding_sample = sent_embedding_sample
sent_embedding_sample = np.asarray(sent_embedding_sample)

In [78]:
embedding_vector_size = 300

tf.reset_default_graph()
training_inputs = tf.placeholder(shape=[None, 300, 100], dtype=tf.float32)  

In [79]:
k = 14
g = tf.Graph()
#with g.as_default():
T = tf.transpose(tf.cast(tf.get_variable("T", initializer=_T), dtype=tf.float32))
M = tf.get_variable("M", [embedding_vector_size, embedding_vector_size], dtype=tf.float32)

In [80]:
zs = attention(training_inputs, M)
rs = reconstruct_embed(k, zs, training_inputs, M, T)

In [81]:
max_val = len(sent_embedding_sample)
n = 5
loss = total_loss(rs, zs, T, training_inputs, max_val, n)

## Adam Optimizer

In [82]:
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

## Tensorflow Seesion

In [ ]:
variables_names = [v.name for v in tf.trainable_variables()]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
    M2 = sess.run(tf.trainable_variables())
    #print("W before: ", M2)
    for _ in range(100):
        ls, _ = sess.run([loss, train_op], feed_dict={training_inputs:sent_embedding_sample})
        #print(ls)
    M1 = sess.run(tf.trainable_variables())
    #print("W after", M1)

Variable:  T:0
Shape:  (14, 300)
Variable:  M:0
Shape:  (300, 300)
Variable:  dense/kernel:0
Shape:  (300, 14)
Variable:  dense/bias:0
Shape:  (14,)


In [1]:
M1[3] == M2[3]

NameError: name 'M1' is not defined

array([[-0.12895024, -0.10986522,  0.07551587, ..., -0.02266879,
        -0.13532795,  0.13203731],
       [-0.07746063, -0.09901691,  0.07792173, ..., -0.10565016,
        -0.02678639, -0.03910506],
       [ 0.12119326, -0.06454878,  0.0271163 , ..., -0.09294194,
         0.12114128, -0.09015402],
       ...,
       [-0.04129875,  0.12668186,  0.03315522, ..., -0.13619281,
         0.00074582, -0.12422828],
       [ 0.00627255,  0.02791442,  0.13213712, ..., -0.05763636,
        -0.12989871,  0.03123641],
       [-0.08403052, -0.09235042, -0.03664988, ..., -0.07433148,
         0.06971644,  0.04787953]], dtype=float32)